### re-evaluate hallu label

In [6]:
import pandas as pd
import json
from tqdm import tqdm

# origin_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/data/VizWiz/train.json"
origin_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/data/VizWiz/val.json"


# sample_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/data/preprocess/vizwiz_train.csv"
sample_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/data/preprocess/vizwiz_val.csv"

sample_pd = pd.read_csv(sample_path)
sample_id_list = sample_pd['question_id'].tolist()

new_save_data = []

with open(origin_path, "r", encoding='utf-8') as f:
    origin_data = json.load(f)
    
for data in tqdm(origin_data):
    image_path  = data['image'].split('.')[0] # remove extention
    
    if image_path in sample_id_list:
        new_save_data.append(data)
        
print(f"Sampled {len(new_save_data)} data with {len(sample_id_list)} origin samples")

save_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/data/preprocess/vizwiz_val.json"
with open(save_path, "w", encoding='utf-8') as f:
    json.dump(new_save_data, f, indent=4)

100%|██████████| 4319/4319 [00:00<00:00, 30353.68it/s]


Sampled 2934 data with 2934 origin samples


In [17]:
import sys 
sys.path.append('/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu')

import json
import os
from tqdm import tqdm

from src.old.probing.generate_vlm_answers import exact_match

# idx, question, image_path, gold_answer, model_answer, hallu_label -> 수정
target_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/instructblip_VizWiz/logits_train_0917_1116.json"
# target_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/instructblip_VizWiz/logits_val_0917_1105.json"

# image, question, answers {"answer_confidence, answer"}
origin_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/data/preprocess/vizwiz_train.json"
# origin_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/data/preprocess/vizwiz_val.json"

dir_path = os.path.dirname(target_path)
file_name = os.path.basename(target_path)
name, ext = os.path.splitext(file_name)
new_path = os.path.join(dir_path, f"{name}_processed{ext}")

with open(target_path, "r", encoding='utf-8') as f:
    target_data = json.load(f)

with open(origin_path, "r", encoding='utf-8') as f:
    origin_data = json.load(f)


new_save_data = []    
for origin, target in tqdm(zip(origin_data, target_data)):
    changed_data = target
    
    new_hallu_label = 1
    model_answer = target['model_answer']
    answers = origin['answers']
    # print(answers)
    gold_list = [ans['answer'] 
                 for ans in answers 
                 if ans.get('answer_confidence', '').lower() != 'no' and 'answer' in ans]
    
    em = [exact_match(model_answer, gold_list)] # correct if 0 | incorrect if 1
    
    # if is correct -> no hallucination
    if 0 in em:
        new_hallu_label = 0
    
    new_model_ans = model_answer.replace("<s>", "").strip()
    
    changed_data['hallu_label'] = new_hallu_label
    changed_data['model_answer'] = new_model_ans
    new_save_data.append(changed_data)
    
with open(new_path, "w", encoding='utf-8') as f:
    json.dump(new_save_data, f, indent=4)
    
print(f"Saved {len(new_save_data)} data with {len(target_data)} for new hallu label")


14991it [00:00, 143863.11it/s]


Saved 14991 data with 14991 for new hallu label


In [4]:
import sys 
sys.path.append('/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu')

import json
import os
from tqdm import tqdm

from src.old.probing.generate_vlm_answers import exact_match

# idx, question, image_path, gold_answer, model_answer, hallu_label -> 수정
target_path_1 = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/qwen2_VizWiz/logits_train_0918_0307.json"
target_path_2 = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/qwen2_VizWiz/logits_train_0918_1358.json"

# image, question, answers {"answer_confidence, answer"}
origin_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/data/preprocess/vizwiz_train.json"
# origin_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/data/preprocess/vizwiz_val.json"

dir_path = os.path.dirname(target_path_1)
file_name = os.path.basename(target_path_1)
name, ext = os.path.splitext(file_name)
new_path = os.path.join(dir_path, f"{name}_processed{ext}")

with open(target_path_1, "r", encoding='utf-8') as f:
    target_data_1 = json.load(f)
    
with open(target_path_2, "r", encoding='utf-8') as f:
    target_data_2 = json.load(f)

with open(origin_path, "r", encoding='utf-8') as f:
    origin_data = json.load(f)

last_idx = target_data_1[-1]['idx']
new_idx = last_idx+1
for data in target_data_2:
    
    append_data = data
    append_data["idx"] = new_idx
    new_idx += 1
    target_data_1.append(append_data)

new_save_data = []    
for origin, target in tqdm(zip(origin_data, target_data_1)):
    changed_data = target
    
    new_hallu_label = 1
    model_answer = target['model_answer']
    answers = origin['answers']
    # print(answers)
    gold_list = [ans['answer'] 
                 for ans in answers 
                 if ans.get('answer_confidence', '').lower() != 'no' and 'answer' in ans]
    
    em = [exact_match(model_answer, gold_list)] # correct if 0 | incorrect if 1
    
    # if is correct -> no hallucination
    if 0 in em:
        new_hallu_label = 0
    
    new_model_ans = model_answer.replace("<s>", "").strip()
    
    changed_data['hallu_label'] = new_hallu_label
    changed_data['model_answer'] = new_model_ans
    new_save_data.append(changed_data)
    
with open(new_path, "w", encoding='utf-8') as f:
    json.dump(new_save_data, f, indent=4)
    
print(f"Saved {len(new_save_data)} data with {len(target_data_1)} for new hallu label")


14991it [00:00, 204324.61it/s]


Saved 14991 data with 14991 for new hallu label


### hallucination classification model 

In [5]:
import json
import pandas as pd

FEATURES = ["logit_mean", "logit_min", "logit_max", "logprob"]
LABEL = "hallu_label"

def load_data(data_path) -> pd.DataFrame:
    if data_path.endswith(".json"):
        with open(data_path, "r", encoding='utf-8') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        
    elif data_path.endswith(".csv"):
        df = pd.read_csv(data_path)
        
    elif data_path.endswith(".jsonl"):
        df = pd.read_json(data_path, lines=True)
        
    else:
        raise ValueError("data path must be one of these: json, csv, jsonl")

    required = FEATURES + [LABEL]
    missing_cols = [c for c in required if c not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")

    # numerize & 결측치 제거
    for c in required:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    
    df = df.dropna(subset=required).reset_index(drop=True)    # 필수 column들만 남김
    
    return df

In [7]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

RANDOM_STATE = 1

def build_single_feature_pipeline() -> Pipeline:
    
    clf = Pipeline(steps=[
        ("scaler", StandardScaler(with_mean=True, with_std=True)),  # 표준화를 통한 안정적인 최적화
        ("sgd", SGDClassifier(
            loss="log_loss",            # logits loss -> binary
            penalty="l2",               # l2 정규화
            alpha=1e-4,                 # 정규화 강도
            max_iter=2000,
            class_weight="balanced",    
            random_state=RANDOM_STATE,
            learning_rate="optimal"
        ))
    ])
    return clf

In [9]:
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
)
import numpy as np
from sklearn.model_selection import train_test_split
from joblib import dump
from tqdm.auto import tqdm

def train_and_eval_single_feature_models(df: pd.DataFrame, save_dir: str):
    os.makedirs(save_dir, exist_ok=True)
    X_all = df[FEATURES].copy()
    y_all = df[LABEL].astype(int).values

    # Stratified split
    X_train, X_test, y_train, y_test = train_test_split(
        X_all, y_all, test_size=0.2, random_state=RANDOM_STATE, stratify=y_all
    )

    results = {}
    iterator = tqdm(FEATURES, desc="Training single-feature SGD models", leave=True)

    for feat in iterator:
        pipe = build_single_feature_pipeline()

        # single feature (n_samples, 1)
        Xtr = X_train[[feat]].values
        Xte = X_test[[feat]].values

        pipe.fit(Xtr, y_train)

        y_prob = pipe.predict_proba(Xte)[:, 1]  # => 1일 것 같은 prob
        y_pred = (y_prob >= 0.5).astype(int)    # trnaslate prob into binary value with threshold 0.5

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        try:
            auc = roc_auc_score(y_test, y_prob)
        except ValueError:
            auc = np.nan 

        cm = confusion_matrix(y_test, y_pred)

        results[feat] = {
            "accuracy": acc,
            "f1": f1,
            "auroc": auc,
            "confusion_matrix": cm.tolist(),
            "report": classification_report(y_test, y_pred, zero_division=0)
        }

        # 모델 저장
        model_path = os.path.join(save_dir, f"sgd_{feat}.joblib")
        dump(pipe, model_path)

        print(f"\n=== Feature: {feat} ===")
        print(f"Accuracy: {acc:.4f} | F1: {f1:.4f} | AUROC: {auc if not np.isnan(auc) else 'NA'}")
        print("Confusion Matrix [[TN, FP],[FN, TP]]:")
        print(cm)
        print("Classification Report:")
        print(results[feat]["report"])
        print(f"Saved: {model_path}")

    return results

In [ ]:
import torch
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
)
import numpy as np
from sklearn.model_selection import train_test_split
from joblib import dump
from tqdm.auto import tqdm
import itertools

FEATURE_NAMES = ["entropy", "diff"]

def train_all_head_classifier_with_single(
    features_tensor: torch.Tensor,
    labels_tensor: torch.Tensor,
    save_dir: str
):
    os.makedirs(save_dir, exist_ok=True)
    
    # 텐서에서 차원 정보 가져오기
    num_samples, num_layers, num_heads, _ = features_tensor.shape
    
    # Stratified split: 전체 데이터가 아닌 인덱스를 분할하여 메모리 효율성 증대
    indices = np.arange(num_samples)
    train_indices, test_indices = train_test_split(
        indices,
        test_size=0.2,
        random_state=RANDOM_STATE,
        stratify=labels_tensor.cpu().numpy() # 계층적 샘플링을 위해 라벨 사용
    )

    y_train = labels_tensor[train_indices].cpu().numpy()
    y_test = labels_tensor[test_indices].cpu().numpy()

    results = {}
    
    # 3중 반복문을 itertools.product로 깔끔하게 처리
    iterator = itertools.product(range(num_layers), range(num_heads), enumerate(FEATURE_NAMES))
    total_classifiers = num_layers * num_heads * len(FEATURE_NAMES)
    
    progress_bar = tqdm(iterator, total=total_classifiers, desc="Training head classifiers")

    for layer_idx, head_idx, (feat_idx, feat_name) in progress_bar:
        
        classifier_name = f"layer_{layer_idx}_head_{head_idx}_{feat_name}"
        progress_bar.set_postfix_str(classifier_name)

        # (num_samples, 1) 형태로 2D 배열 만들기
        X_feature = features_tensor[:, layer_idx, head_idx, feat_idx].reshape(-1, 1)
        
        # 2. Train/Test 데이터 분할
        Xtr = X_feature[train_indices].cpu().numpy()
        Xte = X_feature[test_indices].cpu().numpy()
        
        # 3. 모델 빌드 및 학습
        pipe = build_single_feature_pipeline()
        pipe.fit(Xtr, y_train)

        # 4. 평가
        y_prob = pipe.predict_proba(Xte)[:, 1]
        y_pred = (y_prob >= 0.5).astype(int)
        
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        try:
            auc = roc_auc_score(y_test, y_prob)
        except ValueError:
            auc = np.nan
        cm = confusion_matrix(y_test, y_pred)

        # 5. 결과 저장
        results[classifier_name] = {
            "accuracy": acc,
            "f1": f1,
            "auroc": auc,
            "confusion_matrix": cm.tolist(),
            "report": classification_report(y_test, y_pred, zero_division=0, output_dict=True)
        }
        
        # 6. 모델 파일 저장
        model_path = os.path.join(save_dir, f"sgd_{classifier_name}.joblib")
        dump(pipe, model_path)

    print("\n All head classifier training and evaluation finished.")
    return results

In [9]:
import os
import torch
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
)
import numpy as np
from sklearn.model_selection import train_test_split
from joblib import dump
from tqdm.auto import tqdm
import itertools


RANDOM_STATE = 1 # RANDOM_STATE가 정의되어 있다고 가정

def train_all_head_classifiers_integrated( 
    features_tensor: torch.Tensor,
    labels_tensor: torch.Tensor,
    save_dir: str
):
    print(f"features shape: {features_tensor.shape}")
    print(f"labels shape: {labels_tensor.shape}")
    
    os.makedirs(save_dir, exist_ok=True)
    
    num_samples, num_layers, num_heads, num_features = features_tensor.shape
    
    indices = np.arange(num_samples)
    train_indices, test_indices = train_test_split(
        indices,
        test_size=0.2,
        random_state=RANDOM_STATE,
        stratify=labels_tensor.cpu().numpy()
    )

    y_train = labels_tensor[train_indices].cpu().numpy()
    y_test = labels_tensor[test_indices].cpu().numpy()

    results = {}
    
    iterator = itertools.product(range(num_layers), range(num_heads))
    total_classifiers = num_layers * num_heads
    
    progress_bar = tqdm(iterator, total=total_classifiers, desc="Training integrated head classifiers")

    for layer_idx, head_idx in progress_bar:
        
        classifier_name = f"layer_{layer_idx}_head_{head_idx}_all"
        progress_bar.set_postfix_str(classifier_name)

        # 💡 3. 피처 준비: 마지막 차원을 모두 선택 (num_features=2)
        # 결과 shape: (num_samples, 2)
        X_feature = features_tensor[:, layer_idx, head_idx, :]
        
        # Train/Test 데이터 분할
        Xtr = X_feature[train_indices].cpu().numpy()
        Xte = X_feature[test_indices].cpu().numpy()
        
        # 모델 빌드 및 학습
        pipe = build_single_feature_pipeline()
        pipe.fit(Xtr, y_train)

        # 평가
        y_prob = pipe.predict_proba(Xte)[:, 1]
        y_pred = (y_prob >= 0.5).astype(int)
        
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        try:
            auc = roc_auc_score(y_test, y_prob)
        except ValueError:
            auc = np.nan
        cm = confusion_matrix(y_test, y_pred)

        # 결과 저장
        results[classifier_name] = {
            "accuracy": acc,
            "f1": f1,
            "auroc": auc,
            "confusion_matrix": cm.tolist(),
            "report": classification_report(y_test, y_pred, zero_division=0, output_dict=True)
        }
        
        # 모델 파일 저장
        model_path = os.path.join(save_dir, f"sgd_{classifier_name}.joblib")
        dump(pipe, model_path)

    print("\n All head integrated classifier training and evaluation finished.")
    return results

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch

device = "cuda"
feature_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/llava1.5_VizWiz/train_features_5000.pt"
save_dir = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/llava1.5"

loaded_data = torch.load(feature_path, map_location=device)

features_tensor = loaded_data['features']
labels_tensor = loaded_data['labels']

training_results = train_all_head_classifiers_integrated(features_tensor, labels_tensor, save_dir)

first_key = list(training_results.keys())[0]
print(f"\n--- Sample Result for '{first_key}' ---")
print(f"  AUROC: {training_results[first_key]['auroc']:.4f}")
second_key = list(training_results.keys())[1]

print(f"\n--- Sample Result for '{second_key}' ---")
print(f"  AUROC: {training_results[second_key]['auroc']:.4f}")

features shape: torch.Size([5000, 32, 32, 2])
labels shape: torch.Size([5000])


Training integrated head classifiers:   0%|          | 0/1024 [00:00<?, ?it/s, layer_0_head_0_all]

Training integrated head classifiers: 100%|██████████| 1024/1024 [00:34<00:00, 30.09it/s, layer_31_head_31_all]



 All head integrated classifier training and evaluation finished.

--- Sample Result for 'layer_0_head_0_all' ---
  AUROC: 0.5374

--- Sample Result for 'layer_0_head_1_all' ---
  AUROC: 0.4775


### Llava1.5 classifier

In [10]:

data_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/llava1.5_VizWiz/logits_train_0916_2251.json"
save_dir = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/llava1.5"


df = load_data(data_path)
_ = train_and_eval_single_feature_models(df, save_dir)
    

Training single-feature SGD models: 100%|██████████| 4/4 [00:00<00:00, 22.73it/s]


=== Feature: logit_mean ===
Accuracy: 0.6089 | F1: 0.6431 | AUROC: 0.6413587067913061
Confusion Matrix [[TN, FP],[FN, TP]]:
[[ 769  666]
 [ 507 1057]]
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.54      0.57      1435
           1       0.61      0.68      0.64      1564

    accuracy                           0.61      2999
   macro avg       0.61      0.61      0.61      2999
weighted avg       0.61      0.61      0.61      2999

Saved: /data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/llava1.5/sgd_logit_mean.joblib

=== Feature: logit_min ===
Accuracy: 0.5575 | F1: 0.5466 | AUROC: 0.6039904827254337
Confusion Matrix [[TN, FP],[FN, TP]]:
[[872 563]
 [764 800]]
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.61      0.57      1435
           1       0.59      0.51      0.55      1564

    accuracy                           0.56      299

### Instructblip classifier

In [12]:
data_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/instructblip_VizWiz/logits_train_0917_1116_processed.json"
save_dir = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/instructblip"


df = load_data(data_path)
_ = train_and_eval_single_feature_models(df, save_dir)

Training single-feature SGD models: 100%|██████████| 4/4 [00:00<00:00, 26.00it/s]


=== Feature: logit_mean ===
Accuracy: 0.5765 | F1: 0.6066 | AUROC: 0.6240154710421134
Confusion Matrix [[TN, FP],[FN, TP]]:
[[750 802]
 [468 979]]
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.48      0.54      1552
           1       0.55      0.68      0.61      1447

    accuracy                           0.58      2999
   macro avg       0.58      0.58      0.57      2999
weighted avg       0.58      0.58      0.57      2999

Saved: /data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/instructblip/sgd_logit_mean.joblib

=== Feature: logit_min ===
Accuracy: 0.5519 | F1: 0.3755 | AUROC: 0.598120934532164
Confusion Matrix [[TN, FP],[FN, TP]]:
[[1251  301]
 [1043  404]]
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.81      0.65      1552
           1       0.57      0.28      0.38      1447

    accuracy                           0.55      

### Qwen classifier

In [13]:
data_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/qwen2_VizWiz/logits_train_0918_0307_processed.json"
save_dir = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/qwen"


df = load_data(data_path)
_ = train_and_eval_single_feature_models(df, save_dir)

Training single-feature SGD models: 100%|██████████| 4/4 [00:00<00:00, 25.44it/s]


=== Feature: logit_mean ===
Accuracy: 0.5609 | F1: 0.3522 | AUROC: 0.6755964536738593
Confusion Matrix [[TN, FP],[FN, TP]]:
[[1324 1190]
 [ 127  358]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.53      0.67      2514
           1       0.23      0.74      0.35       485

    accuracy                           0.56      2999
   macro avg       0.57      0.63      0.51      2999
weighted avg       0.80      0.56      0.62      2999

Saved: /data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/qwen/sgd_logit_mean.joblib

=== Feature: logit_min ===
Accuracy: 0.5525 | F1: 0.2877 | AUROC: 0.5733451434851429
Confusion Matrix [[TN, FP],[FN, TP]]:
[[1386 1128]
 [ 214  271]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.55      0.67      2514
           1       0.19      0.56      0.29       485

    accuracy                           0.55      299

## Evaluation

In [15]:
# 필요한 import문 (상단에 선언되어 있는지 확인)
import numpy as np
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
)

def _evaluate(
    pipe, 
    X: np.ndarray, # 입력 feature 행렬
    y_test: np.ndarray,     # 정답 라벨
    threshold: float = 0.5  # 확률 -> 이진 라벨로 바꿀 때 쓸 기준값
    ):
    
    # 확률 예측이 불가능한 모델을 위한 기본값 설정
    y_prob = np.array([np.nan] * len(X))
    y_pred = np.array([np.nan] * len(X))

    if hasattr(pipe, "predict_proba"):
        if hasattr(pipe, "classes_"):
            classes = list(pipe.classes_)
            pos_idx = classes.index(1)
        else:
            pos_idx = 1
        y_prob = pipe.predict_proba(X)[:, pos_idx]
        y_pred = (y_prob >= threshold).astype(int)

    elif hasattr(pipe, "decision_function"):
        scores = pipe.decision_function(X)
        y_prob = 1.0 / (1.0 + np.exp(-scores))
        y_pred = (y_prob >= threshold).astype(int)
    else:
        # 확률/점수 미제공 모델일 경우, 예측만 가능
        y_pred = pipe.predict(X)

    # 평가 지표 계산
    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred, zero_division=0)
    cm  = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=0)
    
    try:
        # y_prob가 np.nan이 아닐 때만 auc 계산
        if not np.isnan(y_prob).all():
            auc = roc_auc_score(y_test, y_prob)
        else:
            auc = np.nan
    except ValueError:
        auc = np.nan

    # y_prob와 y_pred를 반환 딕셔너리에 추가!
    return {
        "accuracy": acc,
        "f1": f1,
        "auroc": auc,
        "confusion_matrix": cm.tolist(),
        "report": report,
        "y_prob": y_prob.tolist(),  # 리스트로 변환하여 반환
        "y_pred": y_pred.tolist(),  # 리스트로 변환하여 반환
    }

In [16]:
from joblib import load

def evaluate_saved_models(
    df: pd.DataFrame,
    model_dir: str,
    threshold: float = 0.5,
    verbose: bool = True
):
    
    y_true = df[LABEL].astype(int).values if LABEL in df.columns else None
    results = {}
    for feat in FEATURES:
        model_path = os.path.join(model_dir, f"sgd_{feat}.joblib")
        if not os.path.exists(model_path):
            if verbose:
                print(f"[SKIP] not found: {model_path}")
            continue

        pipe = load(model_path)
        X = df[[feat]].values  # (n, 1)

        res = _evaluate(pipe, X, y_true, threshold=threshold)
        results[f"sgd_{feat}"] = res

        if verbose:
            print(f"\n=== Loaded: sgd_{feat} ===")
            print(f"Accuracy: {res['accuracy']:.4f} | F1: {res['f1']:.4f} | AUROC: {res['auroc'] if not np.isnan(res['auroc']) else 'NA'}")
            print("Confusion Matrix [[TN, FP],[FN, TP]]:")
            print(np.array(res["confusion_matrix"]))
            print("Classification Report:")
            print(res["report"])
            
        pred_df = pd.DataFrame({
            "idx": df.get("idx", pd.Series(range(len(df)))),
            "y_true": y_true if y_true is not None else pd.NA,
            "y_prob": res.get("y_prob", pd.NA),
            "y_pred": res.get("y_pred", pd.NA),
        })
        results[f"sgd_{feat}"]["predictions"] = pred_df

    return results

In [ ]:
import os
import json
import itertools
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from joblib import load
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
)
from sklearn.linear_model import SGDClassifier # 예시를 위해 import


def _evaluate(
    pipe, 
    X: np.ndarray, 
    y_true: np.ndarray, 
    threshold: float = 0.5
):

    # 확률 예측
    y_prob = pipe.predict_proba(X)[:, 1]
    y_pred = (y_prob >= threshold).astype(int)

    # 평가 지표 계산
    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, zero_division=0)
    cm  = confusion_matrix(y_true, y_pred)
    
    try:
        auc = roc_auc_score(y_true, y_prob)
    except ValueError:
        auc = np.nan


    return {
        "accuracy": acc,
        "f1": f1,
        "auroc": auc,
        "confusion_matrix": cm.tolist(),
        "report": classification_report(y_true, y_pred, zero_division=0, output_dict=True),
    }

/opt/anaconda3/envs/logit_hallu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def evaluate_and_restructure_results(
    features_tensor: torch.Tensor,
    labels_tensor: torch.Tensor,
    model_dir: str,
    output_jsonl_path: str
):
    """
    저장된 모든 헤드 분류기를 불러와 평가하고, 
    (layer, head) 당 한 줄로 결과를 재구성하여 jsonl 파일로 저장합니다.
    """
    num_samples, num_layers, num_heads, _ = features_tensor.shape
    feature_names = ["entropy", "diff"]
    
    print(f"Starting evaluation for {num_layers} layers, {num_heads} heads...")
    
    y_eval = labels_tensor.cpu().numpy()

    with open(output_jsonl_path, 'w') as f:
        # 💡 반복문 구조 변경: layer와 head를 기준으로 순회
        iterator = itertools.product(range(num_layers), range(num_heads))
        total_pairs = num_layers * num_heads
        
        progress_bar = tqdm(iterator, total=total_pairs, desc="Evaluating (layer, head) pairs")

        for layer_idx, head_idx in progress_bar:
            
            # 한 (layer, head) 쌍의 결과를 담을 딕셔너리
            json_line = {
                "layer": layer_idx,
                "head": head_idx,
            }
            
            # 💡 내부에서 두 피처(entropy, diff)에 대해 각각 평가 수행
            for feat_idx, feat_name in enumerate(feature_names):
                model_identifier = f"layer_{layer_idx}_head_{head_idx}_{feat_name}"
                model_path = os.path.join(model_dir, f"sgd_{model_identifier}.joblib")

                if not os.path.exists(model_path):
                    # 모델이 없으면 AUC 값을 NaN으로 기록
                    json_line[f"{feat_name}_auroc"] = np.nan
                    continue

                pipe = load(model_path)
                X_eval = features_tensor[:, layer_idx, head_idx, feat_idx].cpu().numpy().reshape(-1, 1)
                
                eval_results = _evaluate(pipe, X_eval, y_eval)
                
                # 결과를 해당 피처의 키 값으로 딕셔너리에 추가
                json_line[f"{feat_name}_auroc"] = eval_results["auroc"]
            
            # 💡 두 피처에 대한 평가가 끝나면 결과를 한 줄로 파일에 쓰기
            f.write(json.dumps(json_line) + '\n')

    print(f"\n✅ Evaluation finished. Restructured results saved to: {output_jsonl_path}")

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch

device = "cuda"
feature_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/llava1.5_VizWiz/train_features_1000.pt"
save_dir = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/head_auc_llava.jsonl"
model_dir = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/llava1.5"

loaded_data = torch.load(feature_path, map_location=device)

features_tensor = loaded_data['features']
labels_tensor = loaded_data['labels']

evaluate_and_restructure_results(
        features_tensor=features_tensor,
        labels_tensor=labels_tensor,
        model_dir=model_dir,
        output_jsonl_path=save_dir
    )


Starting evaluation for 32 layers, 32 heads...


Evaluating (layer, head) pairs: 100%|██████████| 1024/1024 [00:31<00:00, 32.61it/s]


✅ Evaluation finished. Restructured results saved to: /data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/head_auc_llava.jsonl


### Eval llava classifier

In [18]:
data_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/llava1.5_VizWiz/logits_val_0917_1005.json"
model_dir = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/llava1.5"

df = load_data(data_path)
evaluate_saved_models(df, model_dir)


=== Loaded: sgd_logit_mean ===
Accuracy: 0.6169 | F1: 0.6647 | AUROC: 0.6381514216236439
Confusion Matrix [[TN, FP],[FN, TP]]:
[[ 696  654]
 [ 470 1114]]
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.52      0.55      1350
           1       0.63      0.70      0.66      1584

    accuracy                           0.62      2934
   macro avg       0.61      0.61      0.61      2934
weighted avg       0.61      0.62      0.61      2934


=== Loaded: sgd_logit_min ===
Accuracy: 0.5368 | F1: 0.5152 | AUROC: 0.5872411616161617
Confusion Matrix [[TN, FP],[FN, TP]]:
[[853 497]
 [862 722]]
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.63      0.56      1350
           1       0.59      0.46      0.52      1584

    accuracy                           0.54      2934
   macro avg       0.54      0.54      0.54      2934
weighted avg       0.55      0.54      0.53      2

{'sgd_logit_mean': {'accuracy': 0.6169052488070893,
  'f1': 0.6646778042959427,
  'auroc': 0.6381514216236439,
  'confusion_matrix': [[696, 654], [470, 1114]],
  'report': '              precision    recall  f1-score   support\n\n           0       0.60      0.52      0.55      1350\n           1       0.63      0.70      0.66      1584\n\n    accuracy                           0.62      2934\n   macro avg       0.61      0.61      0.61      2934\nweighted avg       0.61      0.62      0.61      2934\n',
  'y_prob': [0.49988895666187433,
   0.6166191169950125,
   0.3581004420631458,
   0.5531041557650975,
   0.6356607223670399,
   0.39478468940861544,
   0.2125792354684504,
   0.5829181379904452,
   0.6225929341211285,
   0.5830312269415733,
   0.4310695906395566,
   0.5152888277646448,
   0.664794120479399,
   0.4932659321279708,
   0.4803223905315434,
   0.18892671372148,
   0.5701034939625983,
   0.5705870816442725,
   0.5310283481801711,
   0.5912910455319293,
   0.6334647965718962

In [19]:
data_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/instructblip_VizWiz/logits_val_0917_1105_processed.json"
model_dir = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/instructblip"

df = load_data(data_path)
evaluate_saved_models(df, model_dir)


=== Loaded: sgd_logit_mean ===
Accuracy: 0.5971 | F1: 0.6490 | AUROC: 0.6530129551495326
Confusion Matrix [[TN, FP],[FN, TP]]:
[[ 659  815]
 [ 367 1093]]
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.45      0.53      1474
           1       0.57      0.75      0.65      1460

    accuracy                           0.60      2934
   macro avg       0.61      0.60      0.59      2934
weighted avg       0.61      0.60      0.59      2934


=== Loaded: sgd_logit_min ===
Accuracy: 0.5886 | F1: 0.4862 | AUROC: 0.6331459917101913
Confusion Matrix [[TN, FP],[FN, TP]]:
[[1156  318]
 [ 889  571]]
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.78      0.66      1474
           1       0.64      0.39      0.49      1460

    accuracy                           0.59      2934
   macro avg       0.60      0.59      0.57      2934
weighted avg       0.60      0.59      0.57   

{'sgd_logit_mean': {'accuracy': 0.5971370143149284,
  'f1': 0.6490498812351544,
  'auroc': 0.6530129551495326,
  'confusion_matrix': [[659, 815], [367, 1093]],
  'report': '              precision    recall  f1-score   support\n\n           0       0.64      0.45      0.53      1474\n           1       0.57      0.75      0.65      1460\n\n    accuracy                           0.60      2934\n   macro avg       0.61      0.60      0.59      2934\nweighted avg       0.61      0.60      0.59      2934\n',
  'y_prob': [0.5415713495420132,
   0.7983918443228585,
   0.5979625473971355,
   0.6867895695757444,
   0.5441562343768953,
   0.4476318155373263,
   0.16266655149455933,
   0.5647919514150537,
   0.5856603770647151,
   0.6354713595348558,
   0.5102546051125253,
   0.5705047192751529,
   0.6901551500186448,
   0.645189445273292,
   0.4856630786822825,
   0.2268434460585662,
   0.8403102168234058,
   0.6340595637802244,
   0.6208213048679728,
   0.702119156077452,
   0.788087216518228,

In [20]:
data_path = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/qwen2_VizWiz/logits_val_0917_1518_processed.json"
model_dir = "/data3/KJE/code/WIL_DeepLearningProject_2/VLM_Hallu/output/sgd_classifier/qwen"

df = load_data(data_path)
evaluate_saved_models(df, model_dir)


=== Loaded: sgd_logit_mean ===
Accuracy: 0.5232 | F1: 0.3807 | AUROC: 0.6671350602802216
Confusion Matrix [[TN, FP],[FN, TP]]:
[[1105 1271]
 [ 128  430]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.47      0.61      2376
           1       0.25      0.77      0.38       558

    accuracy                           0.52      2934
   macro avg       0.57      0.62      0.50      2934
weighted avg       0.77      0.52      0.57      2934


=== Loaded: sgd_logit_min ===
Accuracy: 0.5521 | F1: 0.3404 | AUROC: 0.5836516297985832
Confusion Matrix [[TN, FP],[FN, TP]]:
[[1281 1095]
 [ 219  339]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.54      0.66      2376
           1       0.24      0.61      0.34       558

    accuracy                           0.55      2934
   macro avg       0.55      0.57      0.50      2934
weighted avg       0.74      0.55      0.60   

{'sgd_logit_mean': {'accuracy': 0.5231765507839128,
  'f1': 0.3806994245241257,
  'auroc': 0.6671350602802216,
  'confusion_matrix': [[1105, 1271], [128, 430]],
  'report': '              precision    recall  f1-score   support\n\n           0       0.90      0.47      0.61      2376\n           1       0.25      0.77      0.38       558\n\n    accuracy                           0.52      2934\n   macro avg       0.57      0.62      0.50      2934\nweighted avg       0.77      0.52      0.57      2934\n',
  'y_prob': [0.4787312298069723,
   0.6663134768284996,
   0.6506485087710778,
   0.5487024086539705,
   0.6867372124857332,
   0.6682727564468092,
   0.5476800230758798,
   0.604370738879958,
   0.6186759558823042,
   0.7057394078933095,
   0.6479186127968449,
   0.6921693267033431,
   0.7711066046027567,
   0.560505428037416,
   0.6067610615346344,
   0.5731951028181119,
   0.6368633895620399,
   0.6242258634806811,
   0.7683119189422541,
   0.2715815079516383,
   0.6480423600839834